In [34]:
import math 

def parse_integer(int_packet):
    reached_end = False
    remaining_packet = int_packet
    binary_total = []

    length_read = 0

    while not reached_end:
        next_5 = remaining_packet[:5]
        length_read += 5
        remaining_packet = remaining_packet[5:]

        binary_total.append(next_5[1:])

        if next_5[0] == "0":
            reached_end = True

    return int("".join(binary_total), 2), length_read

def parse_packet(packet) -> tuple[int, int, int]:

    total_sum = 0
    version_sum = 0

    version = int(packet[:3], 2)
    type_id = int(packet[3:6], 2)

    version_sum += version
    length_read = 6


    # integer literal
    if type_id == 4:
        integer, l_read = parse_integer(packet[6:])
        length_read += l_read
        total_sum += integer
    
    # operator
    else:
        length_type_id = int(packet[6], 2)
        length_read += 1
        result_array_from_children = []

        # 15 bits
        if length_type_id == 0:
            number_of_bits_in_sub_packets = int(packet[7:7+15], 2)
            length_read += 15
            cursor = 7 + 15

            while cursor < number_of_bits_in_sub_packets + 7 + 15:
                v_sum, l_read, total_sum = parse_packet(packet[cursor:])
                version_sum += v_sum
                length_read += l_read
                cursor += l_read
                result_array_from_children.append(total_sum)

        # 11 bits
        else:
            number_of_sub_packets = int(packet[7:7+11], 2)
            length_read += 11
            cursor = 7 + 11

            for n in range(number_of_sub_packets):
                v_sum, l_read, total_sum = parse_packet(packet[cursor:])
                version_sum += v_sum
                cursor += l_read
                length_read += l_read
                result_array_from_children.append(total_sum)

        # operator
        if type_id == 0:
            total_sum = sum(result_array_from_children)
        elif type_id == 1:
            total_sum = math.prod(result_array_from_children)
        elif type_id == 2:
            total_sum = min(result_array_from_children)
        elif type_id == 3:
            total_sum = max(result_array_from_children)
        elif type_id == 5:
            total_sum = 1 if result_array_from_children[0] > result_array_from_children[1] else 0
        elif type_id == 6:
            total_sum = 1 if result_array_from_children[0] < result_array_from_children[1] else 0
        elif type_id == 7:
            total_sum = 1 if result_array_from_children[0] == result_array_from_children[1] else 0
            

    return version_sum, length_read, total_sum

def main(file, type):
    rows = file.read().splitlines()

    for row in rows:
        binary_arr = []
        for char in row:
            binary_arr.append("{0:08b}".format(int(char, 16))[4:])
        binary = "".join(binary_arr)

        v_sum_1, _, total = parse_packet(binary)

        print(f"{type} {row} \nPart 1: {v_sum_1}\nPart 2: {total}\n")

with open("test.txt") as input_file:
    main(input_file, "Test")

with open("input.txt") as input_file:
    main(input_file, "Real")

100111000000000001011010110000101111100011110000
version=4 type_id=7
operator with len= 0 

number_of_bits_in_sub_packets= 22
parsing sub-packet starting at 22
version=5 type_id=4
int= 5 

parsing sub-packet starting at 33
version=7 type_id=4
int= 15 

operator 7 on [5, 15] 
Test 9C005AC2F8F0 
Part 1: 16
Part 2: 0

00000010000011010111100010000000010011010011100101111001011100111101101101011011100100110100110110010010100000001100110010011111010000110000100000000010100001101001010101111101100111110110000010010010001101011001001001100001100111010011001000110000000001000111110000000001000010010111011000111001011101100010100101010011010101100000000001110011011001011011001101110101001110011010110111100110100001111111001100110011111010101000010001101001001000000000101101100110011011110101110111001000010011101000000000100011001011111100001011001001000110111000010010010000000001000001001100110010111010110100000000000110110001000111010110010111011101011001001100110101001100000000010100101100000